Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from sklearn.linear_model import LogisticRegression

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (200000, 28, 28), (200000,))
('Validation set', (10000, 28, 28), (10000,))
('Test set', (10000, 28, 28), (10000,))


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
orig_train_labels = train_labels
orig_test_labels = test_labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (200000, 784), (200000, 10))
('Validation set', (10000, 784), (10000, 10))
('Test set', (10000, 784), (10000, 10))


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [15]:
# Logistic regression
from sklearn import grid_search
train_size = [50, 100, 1000, 5000]
parameters = {'C':[0.2, 0.4, 0.6, 0.8, 1]}
for size in train_size:
    training_data_samples = train_dataset[:size]
    training_label_samples = orig_train_labels[:size]

    logistic = LogisticRegression()
    clf = grid_search.GridSearchCV(logistic, parameters)  
    clf.fit(training_data_samples, training_label_samples)
    print 'Training size = {}. Best C is {}. Accuracy on test set is {}.'.format(size, clf.best_params_,
        clf.score(test_dataset, orig_test_labels))

Training size = 50. Best C is {'C': 0.4}. Accuracy on test set is 0.5941.
Training size = 100. Best C is {'C': 0.4}. Accuracy on test set is 0.7542.
Training size = 1000. Best C is {'C': 0.4}. Accuracy on test set is 0.8399.
Training size = 5000. Best C is {'C': 0.2}. Accuracy on test set is 0.8724.


/Users/tcya/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:417: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=3.
  % (min_labels, self.n_folds)), Warning)


In [29]:
batch_size = 128
hidden_nodes = 1024
C = 0.002

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases_1 = tf.Variable(tf.zeros([hidden_nodes]))
  h_fc1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)


  # Training computation.
  weights_2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  logits = tf.matmul(h_fc1, weights_2) + biases_2
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + \
    C * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2)

In [30]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1000.976379
Minibatch accuracy: 5.5%
Validation accuracy: 28.1%
Minibatch loss at step 500: 232.706772
Minibatch accuracy: 83.6%
Validation accuracy: 62.9%
Minibatch loss at step 1000: 83.907661
Minibatch accuracy: 89.1%
Validation accuracy: 64.7%
Minibatch loss at step 1500: 30.695551
Minibatch accuracy: 89.1%
Validation accuracy: 70.4%
Minibatch loss at step 2000: 11.607265
Minibatch accuracy: 88.3%
Validation accuracy: 73.0%
Minibatch loss at step 2500: 4.595130
Minibatch accuracy: 89.1%
Validation accuracy: 73.7%
Minibatch loss at step 3000: 1.966147
Minibatch accuracy: 89.8%
Validation accuracy: 74.1%
Test accuracy: 93.1%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [96]:
hidden_nodes = 1024
train_subset = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases_1 = tf.Variable(tf.zeros([hidden_nodes]))
  h_fc1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)


  # Training computation.
  weights_2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  logits = tf.matmul(h_fc1, weights_2) + biases_2
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + \
    C * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2)

In [97]:
num_steps = 801

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 995.070557
Training accuracy: 8.0%
Validation accuracy: 25.1%
Loss at step 100: 512.592468
Training accuracy: 100.0%
Validation accuracy: 59.6%
Loss at step 200: 419.621765
Training accuracy: 100.0%
Validation accuracy: 59.6%
Loss at step 300: 343.514954
Training accuracy: 100.0%
Validation accuracy: 59.7%
Loss at step 400: 281.229492
Training accuracy: 100.0%
Validation accuracy: 59.7%
Loss at step 500: 230.231964
Training accuracy: 100.0%
Validation accuracy: 59.7%
Loss at step 600: 188.467773
Training accuracy: 100.0%
Validation accuracy: 59.7%
Loss at step 700: 154.291016
Training accuracy: 100.0%
Validation accuracy: 59.6%
Loss at step 800: 126.320511
Training accuracy: 100.0%
Validation accuracy: 59.7%
Test accuracy: 84.7%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [106]:
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases_1 = tf.Variable(tf.zeros([hidden_nodes]))
  h_fc1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)

  # Dropout
  keep_prob = tf.placeholder("float")
  h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

  # Training computation.
  weights_2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  logits = tf.matmul(h_fc1_drop, weights_2) + biases_2
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + \
    C * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2)

In [107]:
num_steps = 801

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict={keep_prob: 0.5})
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(feed_dict={keep_prob: 1}), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(feed_dict={keep_prob: 1}), test_labels))

Initialized
Loss at step 0: 1095.401733
Training accuracy: 13.5%
Validation accuracy: 31.1%
Loss at step 100: 513.977417
Training accuracy: 97.9%
Validation accuracy: 62.1%
Loss at step 200: 420.529785
Training accuracy: 98.9%
Validation accuracy: 62.3%
Loss at step 300: 343.907410
Training accuracy: 99.7%
Validation accuracy: 62.4%
Loss at step 400: 281.722473
Training accuracy: 99.2%
Validation accuracy: 62.5%
Loss at step 500: 230.440796
Training accuracy: 99.8%
Validation accuracy: 62.4%
Loss at step 600: 188.656738
Training accuracy: 99.4%
Validation accuracy: 62.7%
Loss at step 700: 154.457550
Training accuracy: 99.6%
Validation accuracy: 62.9%
Loss at step 800: 126.422783
Training accuracy: 99.9%
Validation accuracy: 62.6%
Test accuracy: 87.2%


Overfitting decreases a little bit.

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [138]:
batch_size = 128
hidden_nodes = 1024
C = 0.002

graph = tf.Graph()
with graph.as_default():
  global_step = tf.Variable(0, trainable=False) # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 1000, 0.97)
    
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases_1 = tf.Variable(tf.zeros([hidden_nodes]))
  h_fc1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)

  # Dropout
  keep_prob = tf.placeholder("float")
  h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

  # Training computation.
  weights_2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  logits = tf.matmul(h_fc1_drop, weights_2) + biases_2
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + \
    C * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2)) # Regularization
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2)

In [139]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(feed_dict={keep_prob: 1}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict={keep_prob: 1}), test_labels))

Initialized
Minibatch loss at step 0: 1155.927734
Minibatch accuracy: 7.0%
Validation accuracy: 22.0%
Minibatch loss at step 500: 237.027985
Minibatch accuracy: 81.2%
Validation accuracy: 60.2%
Minibatch loss at step 1000: 86.861076
Minibatch accuracy: 79.7%
Validation accuracy: 64.2%
Minibatch loss at step 1500: 32.999458
Minibatch accuracy: 86.7%
Validation accuracy: 67.8%
Minibatch loss at step 2000: 13.134293
Minibatch accuracy: 81.2%
Validation accuracy: 70.1%
Minibatch loss at step 2500: 5.504599
Minibatch accuracy: 88.3%
Validation accuracy: 71.8%
Minibatch loss at step 3000: 2.423176
Minibatch accuracy: 92.2%
Validation accuracy: 72.4%
Test accuracy: 92.2%


In [126]:
batch_size = 128
layer_1 = 512
layer_2 = 256
C = 0.002

graph = tf.Graph()
with graph.as_default():
  global_step = tf.Variable(0, trainable=False) # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 100, 0.97)
    
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, layer_1]))
  biases_1 = tf.Variable(tf.zeros([layer_1]))
  h_fc1 = tf.matmul(tf_train_dataset, weights_1) + biases_1

  # Dropout
  keep_prob = tf.placeholder("float")
  h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

  # Variables.
  weights_2 = tf.Variable(tf.truncated_normal([layer_1, layer_2]))
  biases_2 = tf.Variable(tf.zeros([layer_2]))
  h_fc2 = tf.matmul(h_fc1_drop, weights_2) + biases_2

  # Dropout
  h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

  # Training computation.
  weights_3 = tf.Variable(tf.truncated_normal([layer_2, num_labels]))
  biases_3 = tf.Variable(tf.zeros([num_labels]))
  logits = tf.matmul(h_fc2_drop, weights_3) + biases_3
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + \
    C * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(weights_3)) # Regularization
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
        tf.matmul(
            tf.matmul(
                tf.matmul(tf_valid_dataset, weights_1) 
                + biases_1, weights_2) + biases_2, weights_3) 
            + biases_3)
  test_prediction = tf.nn.softmax(
        tf.matmul(
            tf.matmul(
                tf.matmul(tf_test_dataset, weights_1) 
                + biases_1, weights_2) + biases_2, weights_3) 
            + biases_3)

In [127]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(feed_dict={keep_prob: 1}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict={keep_prob: 1}), test_labels))

Initialized
Minibatch loss at step 0: 9046.275391
Minibatch accuracy: 14.8%
Validation accuracy: 8.2%
Minibatch loss at step 500: nan
Minibatch accuracy: 9.4%
Validation accuracy: 10.0%
Minibatch loss at step 1000: nan
Minibatch accuracy: 10.2%
Validation accuracy: 10.0%
Minibatch loss at step 1500: nan
Minibatch accuracy: 9.4%
Validation accuracy: 10.0%
Minibatch loss at step 2000: nan
Minibatch accuracy: 11.7%
Validation accuracy: 10.0%
Minibatch loss at step 2500: nan
Minibatch accuracy: 4.7%
Validation accuracy: 10.0%
Minibatch loss at step 3000: nan
Minibatch accuracy: 8.6%
Validation accuracy: 10.0%
Test accuracy: 10.0%
